In [74]:
import pandas as pd, numpy, math
documentsFilename = 'estadao_noticias_eleicao.csv'
gabaritoFilename = 'gabarito.csv'
data = pd.read_csv(documentsFilename)
gabarito = pd.read_csv(gabaritoFilename)

In [75]:
#Function to print topFive of documents
def returnTopFive(topFive):
    print "Top five documents"
    returnArray = []
    for document in sorted(topFive, key = topFive.get, reverse=True):
        print "Document: " + str(document) + ", frequence: " + str(topFive[document])    
        returnArray.append(document)
    print returnArray
    return returnArray

#Function to verify if element 's' is nan    
def is_nan(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

In [76]:
#Function to search with binary method
def searchBinary(query):
    searchWords = query.split(" ")
    #The top five document by higher frequences 
    topFive = {}
    for i in range(len(data.values)):
        #Get frequence of document i
        documentID, frequence = frequenceBinary(searchWords, data.values[i]);
        
        #Add document and frequence in topFive dictionary
        if(len(topFive) < 5):
            topFive[documentID]= frequence
        else:
            minKey = None
            for j in topFive.keys():
                if(frequence > topFive[j]):
                    if(minKey == None or topFive[j] < topFive[minKey]):
                        minKey = j
            if(minKey != None):
                topFive[documentID] = frequence;
                del topFive[minKey]
    
    return returnTopFive(topFive)
        
#Function to calculate the frequency of query words in the document
def frequenceBinary(query, document):
    frequenceBinary = {}
    if(is_nan(document[1])):
        document[1] = ""
    if(is_nan(document[2])):
        document[2] = ""
    if(is_nan(document[3])):
        document[3] = ""
    documentWords = document[1].split(" ") + document[2].split(" ") + document[3].split(" ")
    
    for q in query:
        #Iterate on all document words
        for word in documentWords:
            if(q.lower() == word.lower()):
                if(q not in frequenceBinary):
                    frequenceBinary[q] = 1
                    break
    frequence = 0
    for freq in frequenceBinary:
        frequence += frequenceBinary[freq]
    
    return document[5], frequence
   

In [77]:
#Function to search with TF method 
def searchTF(query):
    searchWords = query.split(" ")
    #The top five document by higher frequences 
    topFive = {}
    for i in range(len(data.values)):
        #Get frequence of document i
        documentID, frequence = frequenceTF(searchWords, data.values[i]);
        
        #Add document and frequence in topFive dictionary
        if(len(topFive) < 5):
            topFive[documentID]= frequence
        else:
            minKey = None
            for j in topFive.keys():
                if(frequence > topFive[j]):
                    if(minKey == None or topFive[j] < topFive[minKey]):
                        minKey = j
            if(minKey != None):
                topFive[documentID] = frequence;
                del topFive[minKey]
    
    return returnTopFive(topFive)
        
#Function to calculate the frequency of query words in the document
def frequenceTF(query, document):
    frequence = 0
    if(is_nan(document[1])):
        document[1] = ""
    if(is_nan(document[2])):
        document[2] = ""
    if(is_nan(document[3])):
        document[3] = ""
    documentWords = document[1].split(" ") + document[2].split(" ") + document[3].split(" ")
    
    #Iterate on all document words
    for word in documentWords:
        for q in query:
            if(q.lower() == word.lower()):
                frequence += 1
    
    return document[5], frequence
   

In [78]:
#Function to search with TF method 
def searchTF_IDF(query):
    searchWords = query.split(" ")
    
    #Number of documents in file "data"
    numberOfDocs = len(data.values)
    
    #The top five document by higher frequences 
    topFive = {}
    
    #Calculate the number of documents that query words appear
    dictDocsFrequence = {}
    for i in range(len(data.values)):
        dictDocsFrequence = IDF(searchWords, data.values[i], dictDocsFrequence)
    
    #Calculate IDF of query words
    dictIDF = {}
    for i in dictDocsFrequence.keys(): 
        dictIDF[i] = math.log10((numberOfDocs + 1) / dictDocsFrequence[i])
    
    #Iterate in all documents to calculate frequence of query words
    for i in range(len(data.values)):
        #Get frequence of document i
        documentID, frequence = frequenceTF_IDF(searchWords, data.values[i], dictIDF);
        
        #Add document and frequence in topFive dictionary
        if(len(topFive) < 5):
            topFive[documentID]= frequence
        else:
            minKey = None
            for j in topFive.keys():
                if(frequence > topFive[j]):
                    if(minKey == None or topFive[j] < topFive[minKey]):
                        minKey = j
            if(minKey != None):
                topFive[documentID] = frequence;
                del topFive[minKey]
    
    return returnTopFive(topFive)

#Function to calculate IDF of query words
def IDF(query, document, dictDocsFrequence):
    #Verify if the line is nan
    if(is_nan(document[1])):
        document[1] = " "
    if(is_nan(document[2])):
        document[2] = " "
    if(is_nan(document[3])):
        document[3] = " "
    
    #All the words of documents in array
    documentWords = document[1].split(" ") + document[2].split(" ") + document[3].split(" ")
    
    #Iterate on query words
    for wordQuery in query:
        #Iterate on document words
        for wordDocuments in documentWords:
            #Verify if exist the word in document and change the frequence of documents
            if(wordQuery.lower() == wordDocuments.lower()):
                if(wordQuery in dictDocsFrequence):
                    dictDocsFrequence[wordQuery] = dictDocsFrequence[wordQuery] + 1
                    break
                else:
                    dictDocsFrequence[wordQuery] = 1
                    break

    return dictDocsFrequence

#Function to calculate the frequency of query words in the document multiplicate 
def frequenceTF_IDF(query, document, dictIDF):
    frequence = 0
    
    #Verify if the line is nan
    if(is_nan(document[1])):
        document[1] = ""
    if(is_nan(document[2])):
        document[2] = ""
    if(is_nan(document[3])):
        document[3] = ""
        
    #All the words of documents in array
    documentWords = document[1].split(" ") + document[2].split(" ") + document[3].split(" ")
    
    #Iterate on query words
    for wordQuery in query:
        frequenceWord = 0
        #Iterate on document words
        for wordDocument in documentWords:
            if(wordQuery.lower() == wordDocument.lower()):
                frequenceWord += 1
        #Calculate frequence with IDF
        frequence += frequenceWord * dictIDF[wordQuery]
        
    return document[5], frequence

In [79]:
def search_BM25(query):
    searchWords = query.split(" ")
    
    #Number of documents in file "data"
    numberOfDocs = len(data.values)
    
    #The top five document by higher frequences 
    topFive = {}
    
    #Calculate the number of documents that query words appear
    dictDocsFrequence = {}
    for i in range(len(data.values)):
        dictDocsFrequence = IDF(searchWords, data.values[i], dictDocsFrequence)
        
    #Calculate IDF of query words
    dictIDF = {}
    for i in dictDocsFrequence.keys(): 
        dictIDF[i] = math.log10((numberOfDocs + 1) / dictDocsFrequence[i])
    
    #Iterate in all documents to calculate frequence of query words
    for i in range(len(data.values)):
        #Get frequence of document i
        documentID, frequence = frequenceBM25(searchWords, data.values[i], dictIDF);
        
        #Add document and frequence in topFive dictionary
        if(len(topFive) < 5):
            topFive[documentID]= frequence
        else:
            minKey = None
            for j in topFive.keys():
                if(frequence > topFive[j]):
                    if(minKey == None or topFive[j] < topFive[minKey]):
                        minKey = j
            if(minKey != None):
                topFive[documentID] = frequence;
                del topFive[minKey]
    
    return returnTopFive(topFive)
    
#Function to calculate IDF of query words
def IDF(query, document, dictDocsFrequence):
    #Verify if the line is nan
    if(is_nan(document[1])):
        document[1] = " "
    if(is_nan(document[2])):
        document[2] = " "
    if(is_nan(document[3])):
        document[3] = " "
    
    #All the words of documents in array
    documentWords = document[1].split(" ") + document[2].split(" ") + document[3].split(" ")
    
    #Iterate on query words
    for wordQuery in query:
        #Iterate on document words
        for wordDocuments in documentWords:
            #Verify if exist the word in document and change the frequence of documents
            if(wordQuery.lower() == wordDocuments.lower()):
                if(wordQuery in dictDocsFrequence):
                    dictDocsFrequence[wordQuery] = dictDocsFrequence[wordQuery] + 1
                    break
                else:
                    dictDocsFrequence[wordQuery] = 1
                    break

    return dictDocsFrequence

#Function to calculate the frequency of query words in the document multiplicate 
def frequenceBM25(query, document, dictIDF):
    frequence = 0
    
    #Verify if the line is nan
    if(is_nan(document[1])):
        document[1] = ""
    if(is_nan(document[2])):
        document[2] = ""
    if(is_nan(document[3])):
        document[3] = ""
        
    #All the words of documents in array
    documentWords = document[1].split(" ") + document[2].split(" ") + document[3].split(" ")
    
    #Iterate on query words
    for wordQuery in query:
        frequenceWord = 0
        #Iterate on document words
        for wordDocument in documentWords:
            if(wordQuery.lower() == wordDocument.lower()):
                frequenceWord += 1
        #Calculate frequence with IDF
        frequence += frequenceWord * dictIDF[wordQuery]
        
    return document[5], frequence
    

In [96]:
def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.
    This function computes the average prescision at k between two lists of
    items.
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    print predicted
    print actual
    print type(predicted)
    print type(actual)
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.
    This function computes the mean average prescision at k between two lists
    of lists of items.
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    print type(predicted)
    print type(actual)
    return numpy.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [97]:
busca_binaria = searchBinary("segundo turno")
busca_TF = searchTF("segundo turno")
busca_TF_IDF = searchTF_IDF("segundo turno")

mapk(gabarito.values[0][2], busca_binaria, k=5)

mapk(gabarito.values[1][2], busca_TF, k=5)

mapk(gabarito.values[2][2], busca_TF_IDF, k=5)

#mapk(gabarito.bm25, suaresposta.bm25, k=5)

Top five documents
Document: 69, frequence: 2
Document: 102, frequence: 2
Document: 7, frequence: 2
Document: 111, frequence: 2
Document: 92, frequence: 2
[69, 102, 7, 111, 92]
Top five documents
Document: 2744, frequence: 31
Document: 7, frequence: 27
Document: 2112, frequence: 17
Document: 2388, frequence: 17
Document: 4931, frequence: 15
[2744, 7, 2112, 2388, 4931]
Top five documents
Document: 2744, frequence: 19.1301175722
Document: 7, frequence: 11.3938724518
Document: 1917, frequence: 8.78690753572
Document: 2112, frequence: 8.38357249516
Document: 4931, frequence: 7.78151250384
[2744, 7, 1917, 2112, 4931]
<type 'list'>
<type 'str'>
69
[
<type 'int'>
<type 'str'>


TypeError: object of type 'int' has no len()

In [86]:
print gabarito.values
print gabarito.values[0][2]

[['segundo turno' '[1062, 1942, 2161, 2078, 2073]'
  '[2048, 1, 2049, 2050, 4096]' '[2744, 7, 2112, 7672, 2388]'
  '[2744, 2112, 7672, 1235, 2388]' '[2744, 2112, 7672, 2388, 2178]']
 ['lava jato' '[616, 164, 1734, 163, 6716]' '[3, 13, 15, 27, 6177]'
  '[163, 353, 2807, 127, 359]' '[163, 353, 2807, 127, 359]'
  '[163, 353, 2807, 127, 359]']
 ['projeto de lei' '[2853, 275, 978, 7092, 3171]'
  '[3584, 6145, 8194, 8706, 6660]' '[7, 3942, 7017, 1250, 6942]'
  '[2232, 6461, 2853, 3171, 3942]' '[2232, 6461, 3171, 2853, 3170]']
 ['compra de voto' '[2200, 8615, 2265, 7746, 82]'
  '[7424, 2178, 6531, 5122, 2311]' '[3942, 7017, 5129, 2047, 748]'
  '[7343, 7293, 6791, 3942, 2047]' '[7343, 7293, 6791, 7329, 8615]']
 ['minist\xc3\xa9rio p\xc3\xbablico' '[64, 6652, 164, 6550, 8615]'
  '[8194, 7, 4104, 8201, 4109]' '[6798, 8018, 6244, 6965, 6550]'
  '[6798, 8018, 6244, 6965, 6550]' '[6798, 8018, 6244, 6965, 6550]']]
[2048, 1, 2049, 2050, 4096]


In [ ]:
segundo turno,
google: "[1062, 1942, 2161, 2078, 2073]",
Binary: "[2048, 1, 2049, 2050, 4096]",
TF: "[2744, 7, 2112, 7672, 2388]",
TD-IDF: "[2744, 2112, 7672, 1235, 2388]",
BM25: "[2744, 2112, 7672, 2388, 2178]"